In [37]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
!pip install xlsxwriter
import xlsxwriter

In [38]:
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AlgoTrading/sp_500_stocks.csv')

In [39]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [40]:
symbol = 'AAPL'


api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 90229311,
 'avg30Volume': 92654293,
 'beta': 1.186030706419308,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 110.02,
 'day30ChangePercent': 0.06307428574511825,
 'day50MovingAvg': 120.28,
 'day5ChangePercent': 0.0024896456067901407,
 'dividendYield': 0.006722788282956528,
 'employees': 0,
 'exDividendDate': '2020-10-29',
 'float': 0,
 'marketcap': 2152904701018,
 'maxChangePercent': 47.36405142918944,
 'month1ChangePercent': 0.06416951079141893,
 'month3ChangePercent': 0.09172739232400891,
 'month6ChangePercent': 0.4066414357777222,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 36.932986957384934,
 'sharesOutstanding': 17029842480,
 'ttmDividendRate': 0.8090772220529272,
 'ttmEPS': 3.43,
 'week52change': 0.8471129187133951,
 'week52high': 140.41,
 'week52low': 57.33,
 'year1ChangePercent': 0.8648816786484822,
 'year2ChangePercent': 2.038370551661351,
 'year5ChangePercent': 3.621596781128758,
 'ytdChangePercent': 0.7147783933883343}

In [41]:
data['year1ChangePercent']

0.8648816786484822

In [42]:
def chunks(lst, n):
  for i in range(0, len(lst),n):
    yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Stock Price', 'One Year Price Return', 'Num Shares to Buy']

In [43]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
  batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series([
                 symbol,
                 data[symbol]['price'],
                 data[symbol]['stats']['year1ChangePercent'],
                 'N/A'  
        ], index=my_columns),
        ignore_index = True
    )

final_dataframe

,Ticker,Stock Price,One Year Price Return,Num Shares to Buy
0,A,123.45,0.466065,N/A
1,AAL,17.88,-0.33408,N/A
2,AAP,162.64,0.020757,N/A
3,AAPL,125.22,0.853419,N/A
4,ABBV,108.89,0.321852,N/A
...,...,...,...,...
500,YUM,110.84,0.0950749,N/A
501,ZBH,143.28,0.013801,N/A
502,ZBRA,384.51,0.475139,N/A
503,ZION,43.23,-0.114202,N/A


In [44]:
final_dataframe.sort_values('One Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

In [45]:
#calculating the number of shares to buy
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Enter the size of your portfolio: ')

  try:
    val = float(portfolio_size)
    print(val)
  except:
    print('That is not a number')
    portfolio_input()

portfolio_input()

Enter the size of your portfolio: 1000000
1000000.0


In [46]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
  final_dataframe.loc[i, 'Num Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Stock Price,One Year Price Return,Num Shares to Buy
0,78,CARR,38.90,2.15378,514
1,345,NVDA,541.48,1.4678,36
2,275,LB,39.88,1.38904,501
3,31,AMD,92.46,1.38286,216
4,23,ALB,142.99,1.25526,139
5,387,PYPL,220.85,1.09615,90
6,179,FCX,25.81,1.03848,774
7,339,NOW,562.29,1.03723,35
8,388,QCOM,144.65,0.911814,138
9,288,LRCX,504.20,0.878037,39


In [47]:
#Lets utilize high quality momentum

hqm_columns = [
               'Ticker',
               'Price',
               'Number of Shares to Buy',
               'One-Year Price Return',
               'One-Year Return Percentile',
               'Six-Month Price Return',
               'Six-Month Return Percentile',
               'Three-Month Price Return',
               'Three-Month Return Percentile',
               'One-Month Price Return',
               'One-Month Return Percentile',
               'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)


for symbol_string in symbol_strings:
  batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):


    hqm_dataframe = hqm_dataframe.append(
        pd.Series([
               symbol,
               data[symbol]['price'],
               'n/a',
               data[symbol]['stats']['year1ChangePercent'],
               'n/a',
               data[symbol]['stats']['month6ChangePercent'],
               'n/a',
               data[symbol]['stats']['month3ChangePercent'],
               'n/a',
               data[symbol]['stats']['month1ChangePercent'],
               'n/a',
               'n/a'
        ], index=hqm_columns),
        ignore_index=True
    )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.69,n/a,0.458837,n/a,0.340147,n/a,0.215641,n/a,0.0807982,n/a,n/a
1,AAL,17.31,n/a,-0.334319,n/a,0.0596156,n/a,0.389958,n/a,0.455067,n/a,n/a
2,AAP,158.52,n/a,0.0198503,n/a,0.131873,n/a,0.0329212,n/a,0.0188695,n/a,n/a
3,AAPL,125.74,n/a,0.859929,n/a,0.420626,n/a,0.0897651,n/a,0.0632584,n/a,n/a
4,ABBV,108.29,n/a,0.322188,n/a,0.140276,n/a,0.221353,n/a,0.0904002,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.05,n/a,0.0969833,n/a,0.148898,n/a,0.159647,n/a,0.0391035,n/a,n/a
501,ZBH,148.47,n/a,0.013748,n/a,0.0939218,n/a,0.0555458,n/a,-0.0277939,n/a,n/a
502,ZBRA,378.52,n/a,0.473962,n/a,0.410719,n/a,0.516858,n/a,0.0905985,n/a,n/a
503,ZION,42.94,n/a,-0.118894,n/a,0.182149,n/a,0.39778,n/a,0.0940286,n/a,n/a


In [48]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
      if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
      change_col = f'{time_period} Price Return'
      percentile_col = f'{time_period} Return Percentile'
      hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.69,n/a,0.458837,0.885149,0.340147,0.811881,0.215641,0.635644,0.0807982,0.685149,n/a
1,AAL,17.31,n/a,-0.334319,0.0415842,0.0596156,0.271287,0.389958,0.885149,0.455067,0.988119,n/a
2,AAP,158.52,n/a,0.0198503,0.380198,0.131873,0.425743,0.0329212,0.229703,0.0188695,0.421782,n/a
3,AAPL,125.74,n/a,0.859929,0.974257,0.420626,0.90099,0.0897651,0.376238,0.0632584,0.627723,n/a
4,ABBV,108.29,n/a,0.322188,0.79802,0.140276,0.441584,0.221353,0.649505,0.0904002,0.718812,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.05,n/a,0.0969833,0.516832,0.148898,0.455446,0.159647,0.512871,0.0391035,0.516832,n/a
501,ZBH,148.47,n/a,0.013748,0.370297,0.0939218,0.358416,0.0555458,0.29703,-0.0277939,0.174257,n/a
502,ZBRA,378.52,n/a,0.473962,0.893069,0.410719,0.89505,0.516858,0.952475,0.0905985,0.720792,n/a
503,ZION,42.94,n/a,-0.118894,0.178218,0.182149,0.550495,0.39778,0.891089,0.0940286,0.726733,n/a


In [55]:
#HQM score: arithmatic mean of the 4 momentum percentile scores that we calculated in the last section
from statistics import mean

for row in hqm_dataframe.index:
  momentum_percentiles = []
  for time_period in time_periods:
    momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = float(mean(momentum_percentiles))

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.69,n/a,0.458837,0.885149,0.340147,0.811881,0.215641,0.635644,0.0807982,0.685149,0.754455
1,AAL,17.31,n/a,-0.334319,0.0415842,0.0596156,0.271287,0.389958,0.885149,0.455067,0.988119,0.546535
2,AAP,158.52,n/a,0.0198503,0.380198,0.131873,0.425743,0.0329212,0.229703,0.0188695,0.421782,0.364356
3,AAPL,125.74,n/a,0.859929,0.974257,0.420626,0.90099,0.0897651,0.376238,0.0632584,0.627723,0.719802
4,ABBV,108.29,n/a,0.322188,0.79802,0.140276,0.441584,0.221353,0.649505,0.0904002,0.718812,0.65198
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.05,n/a,0.0969833,0.516832,0.148898,0.455446,0.159647,0.512871,0.0391035,0.516832,0.500495
501,ZBH,148.47,n/a,0.013748,0.370297,0.0939218,0.358416,0.0555458,0.29703,-0.0277939,0.174257,0.3
502,ZBRA,378.52,n/a,0.473962,0.893069,0.410719,0.89505,0.516858,0.952475,0.0905985,0.720792,0.865347
503,ZION,42.94,n/a,-0.118894,0.178218,0.182149,0.550495,0.39778,0.891089,0.0940286,0.726733,0.586634


In [62]:
#Selecting the 50 best momentum stocks

hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,level_0,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,179,FCX,25.59,n/a,1.06542,0.988119,1.18616,0.99802,0.5771,0.968317,0.257406,0.940594,0.973762
1,1,275,LB,40.89,n/a,1.33337,0.994059,1.38549,1,0.485817,0.944554,0.20911,0.90297,0.960396
2,2,288,LRCX,497.30,n/a,0.88695,0.980198,0.664405,0.974257,0.673051,0.978218,0.211743,0.906931,0.959901
3,3,23,ALB,138.70,n/a,1.25395,0.992079,0.70981,0.978218,0.526189,0.954455,0.211529,0.90495,0.957426
4,4,29,AMAT,88.70,n/a,0.577816,0.930693,0.485074,0.926733,0.599364,0.972277,0.278186,0.952475,0.945545
5,5,495,XLNX,148.37,n/a,0.616599,0.946535,0.533057,0.948515,0.469858,0.938614,0.183356,0.887129,0.930198
6,6,324,MU,72.58,n/a,0.521799,0.906931,0.373321,0.855446,0.591146,0.970297,0.294536,0.962376,0.923762
7,7,462,URI,249.72,n/a,0.592173,0.934653,0.575089,0.956436,0.424271,0.910891,0.161237,0.865347,0.916832
8,8,24,ALGN,515.14,n/a,0.875953,0.978218,0.900367,0.990099,0.556081,0.958416,0.0963777,0.738614,0.916337
9,9,388,QCOM,150.39,n/a,0.911949,0.984158,0.744486,0.984158,0.408304,0.90297,0.119533,0.792079,0.915842


In [63]:
portfolio_input(
    
)

Enter the size of your portfolio: 1000000
1000000.0


In [70]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
  hqm_dataframe.loc[i,'Number of Shares to buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])

del hqm_dataframe['level_0']
del hqm_dataframe['index']


In [72]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index=False)

In [73]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)